# Unemployment in India and Credit Card Spendings

We have data from two context:
- Unemployment in India
- Credit Card Spendings

In [465]:
import pandas as pd
from ydata_profiling import ProfileReport

## Load the data

In [466]:

df_spendings = pd.read_csv(r'kaggle\input\Credit_Card_Transactions_date_changed.csv')
df_unemployment = pd.read_csv(r'.\kaggle\input\Unemployment in India.csv')

## Profiling the data

In [467]:
# Unployment data
profile_unemployment = ProfileReport(df_unemployment, title='Unemployment in India', explorative=True)

# Spendings data
profile_spendings = ProfileReport(df_spendings, title='Credit card transactions - India - Simple', explorative=True)

In [ ]:
# Save the reports
profile_unemployment.to_file(r'.\profiles\unemployment_data_profile.html')
profile_spendings.to_file(r'.\profiles\spendings_data_profile.html')

## Exploring the data

In [ ]:
df_unemployment.head()

In [ ]:
df_spendings.head()

#### Integrity constraints 

**Unemployment data**:  

- We should have unique month-year for each row  
- Percentage should be between 0 and 100  
- There should be only Urban and Rural as values for Area

**Spendings data**:

- Amount should be positive and greater than 0 and not null
- Date should be filled
- City should be filled to allow integration with unemployment data

Missing values 

In [ ]:
df_spendings.columns

In [ ]:
df_spendings.shape[0]

In [ ]:
import numpy as np

# Specify columns and range for missing fraction (0-10%)
target_columns = ['City', 'Date', 'Amount']
max_missing_fraction = 0.1  # Maximum 20% missing values

for column in target_columns:
    # Generate a random missing fraction between 0 and 20%
    missing_fraction = np.random.uniform(0, max_missing_fraction)
    
    # Calculate number of missing values for this column
    n_missing = int(missing_fraction * len(df_spendings))
    
    # Ensure that the fraction doesn't exceed the intended max missing count
    n_missing = min(n_missing, int(max_missing_fraction * len(df_spendings)))
    print(n_missing)

    # Randomly select indices to set as NaN
    missing_indices = np.random.choice(df_spendings.index, n_missing, replace=False)
    
    # Set selected indices in the column to NaN
    df_spendings.loc[missing_indices, column] = np.nan

# Display the number of missing values per column for confirmation
print("Missing values introduced:")
print(df_spendings.isna().sum())


In [ ]:
df_spendings.info()

In [ ]:
# Spendings data
profile_spendings_missing_values = ProfileReport(df_spendings, title='Credit card transactions - India - Simple', explorative=True)
profile_spendings_missing_values.to_file(r'.\profiles\spendings_data_profile_missing_values.html')

In [ ]:
df_spendings.head(10)

how to handle missing values? 
1) add random value 
2) mean value
3) interquartile range
4) delete row  

pros & cons for each one:
1) pros: easy to apply with numpy 
   cons: without a range, I can add

2) pros:    
   cons:

3) pros:
   cons:

4) pros:
   cons:     

In [ ]:
df_spendings[df_spendings['City'] == "Greater Mumbai, India	"][['Date']]

Missing values in Unemployment Dataset

In [ ]:
df_unemployment.head(30)


In [ ]:
df_unemployment['Region'].value_counts()

In [ ]:
df_unemployment[df_unemployment['Region'] == 'Delhi']

after watching this, we realized that we have 2 different estimated unemployment rate according to the area. However, since we're studying the credit card transactions, we believe that the best choice is to look only when the area is urban

In [481]:
df_unemployment = df_unemployment[df_unemployment['Area']=='Urban']

In [ ]:
df_unemployment

In [ ]:
df_unemployment['Region'].value_counts()

In [484]:
df_unemployment.columns = df_unemployment.columns.str.strip()


In [485]:
columns_to_remove = ['Area', 'Estimated Labour Participation Rate (%)', 'Estimated Employed', 'Frequency']
df_unemployment = df_unemployment.drop(columns=columns_to_remove, errors='ignore')

In [ ]:
df_unemployment = df_unemployment.applymap(lambda x: x.replace('\n', '') if isinstance(x, str) else x)


In [ ]:
df_unemployment.columns

In [ ]:
df_unemployment['Region'].value_counts()

In [ ]:
df_unemployment.columns

In [490]:

required_dates = [
    '31-05-2019', '30-06-2019', '31-07-2019', '31-08-2019',
    '30-09-2019', '31-10-2019', '30-11-2019', '31-12-2019',
    '31-01-2020', '29-02-2020', '31-03-2020', '30-04-2020',
    '31-05-2020', '30-06-2020'
]

#

In [ ]:
df_unemployment[df_unemployment['Region']=='Jammu & Kashmir']

In [ ]:


# Define required dates
required_dates = [
    '31-05-2019', '30-06-2019', '31-07-2019', '31-08-2019',
    '30-09-2019', '31-10-2019', '30-11-2019', '31-12-2019',
    '31-01-2020', '29-02-2020', '31-03-2020', '30-04-2020',
    '31-05-2020', '30-06-2020'
]

# Convert required dates to datetime format
required_dates = pd.to_datetime(required_dates, format='%d-%m-%Y')

# Convert the 'Date' column to datetime format in the dataset
df_unemployment['Date'] = pd.to_datetime(df_unemployment['Date'], errors='coerce')

# Get the unique regions
unique_regions = df_unemployment['Region'].unique()

# Initialize a list to collect rows with missing dates
missing_dates_rows = []

# Check for missing dates in each region and add rows as needed
for region in unique_regions:
    region_data = df_unemployment[df_unemployment['Region'] == region]
    region_dates = region_data['Date'].unique()
    
    # Identify missing dates
    missing_dates = set(required_dates) - set(region_dates)
    
    # Create new rows for missing dates
    for missing_date in missing_dates:
        missing_dates_rows.append({
            'Region': region,
            'Date': missing_date,
            'Frequency': None,
            'Estimated Unemployment Rate (%)': None,
            'Estimated Employed': None,
            'Estimated Labour Participation Rate (%)': None,
            'Area': None
        })

# Convert missing dates list to a DataFrame and append to the original data
missing_dates_df = pd.DataFrame(missing_dates_rows)
df_unemployment = pd.concat([df_unemployment, missing_dates_df], ignore_index=True)

# Sort the data by Region and Date for better readability
df_unemployment = df_unemployment.sort_values(by=['Region', 'Date']).reset_index(drop=True)



In [ ]:
df_unemployment.shape[0]

In [ ]:
df_unemployment['Region'].value_counts()

In [ ]:
df_unemployment[df_unemployment['Region']=='Jammu & Kashmir']


In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Load your data into df_unemployment (assuming this DataFrame is already defined)
# df_unemployment = pd.read_csv("your_data.csv")  # Uncomment and specify your file if needed

# Convert Date to datetime format
df_unemployment['Date'] = pd.to_datetime(df_unemployment['Date'], errors='coerce')

# Sort by Region and Date to maintain order
df_unemployment = df_unemployment.sort_values(by=['Region', 'Date']).reset_index(drop=True)

# Initialize the Linear Regression model
model = LinearRegression()

# Create a copy of the data to store imputed values
df_imputed = df_unemployment.copy()

# Store metrics results
mae_list = []
r2_list = []

# Apply imputation by region
regions = df_unemployment['Region'].unique()

for region in regions:
    # Select data for the current region
    region_data = df_unemployment[df_unemployment['Region'] == region]
    
    # Extract indices of missing and non-missing values
    missing_indices = region_data['Estimated Unemployment Rate (%)'].isna()
    non_missing_indices = ~missing_indices

    # Continue only if there are at least two known values to fit the model
    if non_missing_indices.sum() > 1:  # We need at least two points to fit a linear model
        # Prepare data for regression
        X = region_data.loc[non_missing_indices, 'Date'].map(pd.Timestamp.toordinal).values.reshape(-1, 1)
        y = region_data.loc[non_missing_indices, 'Estimated Unemployment Rate (%)'].values
        
        # Perform a 70-30 train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Fit the model on training data
        model.fit(X_train, y_train)
        
        # Predict on test data
        y_pred_test = model.predict(X_test)
        
        # Calculate metrics on test data
        mae = mean_absolute_error(y_test, y_pred_test)
        r2 = r2_score(y_test, y_pred_test)
        
        # Append metrics to lists
        mae_list.append(mae)
        r2_list.append(r2)
        
        # Predict missing values if any
        if missing_indices.sum() > 0:
            X_missing = region_data.loc[missing_indices, 'Date'].map(pd.Timestamp.toordinal).values.reshape(-1, 1)
            y_pred_missing = model.predict(X_missing)
            
            # Assign predictions only for rows where `missing_indices` is True in `df_imputed`
            df_imputed.loc[region_data.index[missing_indices], 'Estimated Unemployment Rate (%)'] = y_pred_missing

# Calculate and display average metrics
print(f"Mean Absolute Error (MAE): {np.mean(mae_list):.4f}")
print(f"R-squared (R²): {np.mean(r2_list):.4f}")


In [ ]:
df_imputed.isna().sum()

In [ ]:
df_imputed.info()

Impute missing values in spendings data from india

In [499]:
df_spendings_ma = df_spendings.copy()

In [ ]:
df_spendings_ma

In [ ]:
df_spendings_ma.isna().sum()

In [502]:
columns_to_remove = ['Gender', 'Exp Type', 'Card Type', 'df_index']
df_spendings_ma = df_spendings_ma.drop(columns=columns_to_remove, errors='ignore')

In [ ]:
df_spendings_ma


In [504]:
df_spendings_ma['City'] = df_spendings_ma['City'].str.split(',').str[0]

In [ ]:
df_spendings_ma

In [ ]:
df_spendings_ma['City'].value_counts().head(20)

In [507]:
import numpy as np

# Define the top cities
top_cities = df_spendings_ma['City'].value_counts().index[:4]



In [ ]:
top_cities


In [509]:
# Generate random choices only for NaN values in the 'City' column
random_cities = np.random.choice(top_cities, size=df_spendings_ma['City'].isna().sum())

# Assign these random cities to the NaN values in the 'City' column only
df_spendings_ma.loc[df_spendings['City'].isna(), 'City'] = random_cities

In [ ]:
df_spendings_ma.isna().sum()

In [ ]:
df_spendings_ma

In [ ]:
df_spendings_ma.isna().sum()

In [ ]:
df_spendings_ma

In [514]:
# Calculate the overall mode of the Date column as a fallback
overall_mode_date = df_spendings_ma['Date'].mode()[0] if not df_spendings_ma['Date'].mode().empty else None

# Fill missing dates with the mode for each city or with the overall mode if a city has no dates
df_spendings_ma['Date'] = df_spendings_ma.groupby('City')['Date'].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else overall_mode_date)
)


In [ ]:
df_spendings_ma.isna().sum()

In [ ]:
df_spendings_ma

In [ ]:
df_cleaned = df_spendings_ma.dropna(subset=['Amount'])

# Step 2: Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df_cleaned['Amount'].quantile(0.25)
Q3 = df_cleaned['Amount'].quantile(0.75)

# Step 3: Calculate IQR
IQR = Q3 - Q1

# Step 4: Fill missing values in 'Amount' with the IQR
df_spendings_ma['Amount'] = df_spendings_ma['Amount'].fillna(IQR)

# Display the result to confirm the missing values have been filled
print(df_spendings_ma)


In [ ]:
df_spendings_ma.isna().sum()

add region in spending dataset 

In [519]:
df_india_cities = pd.read_csv("kaggle\input\cities_state_india.csv")

In [520]:
df_india_cities = df_india_cities.drop(columns='Unnamed: 0')

In [ ]:
df_india_cities = df_india_cities.loc[df_india_cities.groupby('City')['Population (2011)'].idxmax()]

# Reset index for a clean output
df_india_cities = df_india_cities.reset_index(drop=True)

# Display the resulting DataFrame
df_india_cities

In [522]:
# Define a dictionary of common alternate city names for normalization
city_name_mapping = {
    "Bengaluru": "Bangalore"
    # Add more mappings as needed
}

# Function to normalize city names
def normalize_city_name(city_name):
    # Check if the city name has an alternate in the dictionary
    return city_name_mapping.get(city_name, city_name)

# Update the get_region function to include normalization
def get_region_with_normalization(city_name):
    # Normalize the city name
    normalized_city = normalize_city_name(city_name)
    
    # Match based on "contains" logic
    for index, row in df_india_cities.iterrows():
        if row['City'].lower() in normalized_city.lower() or normalized_city.lower() in row['City'].lower():
            return row['State or union territory']
    return None

# Apply the updated function to add a new 'Region' column to df_spending_region
df_spendings_ma['Region'] = df_spendings_ma['City'].apply(get_region_with_normalization)



In [523]:
df_teste = df_spendings_ma[df_spendings_ma.isna().any(axis=1)]